In [2]:
import pandas as pd
import xlrd

## Load TPM data matrix

In [3]:
cancer_tpm = pd.read_csv('data/GSE131512_cancerTPM.txt', sep='\t')

In [4]:
cancer_tpm.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96
ENSG00000000003,7.071605,60.610797,58.255903,29.917356,24.500322,25.311091,37.394452,15.354658,24.839014,24.591295,...,14.134044,10.803910,6.068982,12.714350,15.983392,0.000000,38.189128,15.608352,51.519267,30.805593
ENSG00000000005,13.279391,47.424080,60.455497,6.482332,53.675826,31.686900,32.409716,0.000000,48.976056,46.178684,...,0.000000,12.172853,136.759411,28.650721,0.000000,0.000000,5.976111,6.280734,241.863225,0.000000
ENSG00000000419,0.000000,0.000000,23.040206,8.646690,20.456404,42.266702,21.615428,11.538256,18.665274,76.996356,...,79.657691,8.118597,0.000000,0.000000,0.000000,179.178412,15.942897,0.000000,0.000000,0.000000
ENSG00000000457,6.212355,2.218590,12.120963,6.065120,16.142519,37.059356,7.580945,0.000000,0.000000,21.603278,...,18.624986,15.660412,33.988677,0.000000,36.858404,83.788404,18.172309,2.938248,14.547636,10.148436
ENSG00000000460,34.038592,35.828348,55.926653,10.494273,14.482668,19.236768,13.117055,4.667898,18.877984,12.459809,...,17.903438,19.706680,23.062537,1.932615,28.344382,3.020339,27.411791,1.694651,3.729079,7.804221


In [5]:

ensg_to_hgnc = pd.read_csv('reference_data/ENSG_to_HGNC.txt', sep='\t')
ensg_to_hgnc.index = ensg_to_hgnc['Gene stable ID']
ensg_to_hgnc_dict = ensg_to_hgnc['Gene name'].to_dict()


In [6]:

# Replace ENSG with HGNC for better readability 
new_indices = []
for i in cancer_tpm.index:
    hgnc = ensg_to_hgnc_dict.get(i, None)
    if not hgnc:
        new_index = i
    else:
        new_index = hgnc
    if new_index in new_indices:
        new_index = '{}({})'.format(new_index, i)
    new_indices.append(new_index)


KeyboardInterrupt: 

In [ ]:
# Make multi-index with both gene names and ENSG IDs

multi_index_lists = [cancer_tpm.index, new_indices]
multi_index_tuples = list(zip(*multi_index_lists))

cancer_tpm.index = pd.MultiIndex.from_tuples(multi_index_tuples, names=['ENSG', 'HGNC'])
cancer_tpm.to_csv('data/cancer_tpm_updated_indices.tsv', sep='\t', header=True)

In [7]:
cancer_tpm = pd.read_csv('data/cancer_tpm_updated_indices.tsv', sep='\t', index_col=[0,1]).transpose()
cancer_tpm.head()

ENSG,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000283098,ENSG00000283101,ENSG00000283103,ENSG00000283108,ENSG00000283110,ENSG00000283117,ENSG00000283118,ENSG00000283122,ENSG00000283123,ENSG00000283125
HGNC,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL132857.2,BMS1P21,AC010642.2,AC011451.3,AC141257.4,AC004949.1,AC009951.5,HYMAI,RARRES2P11,AC022726.2
C1,7.071605,13.279391,0.000000,6.212355,34.038592,9.231355,32.811263,19.136966,12.631348,11.220057,...,12.308474,0.0,4.430132,0.000000,0.0,6.856902,16.599238,22.357146,0.0,19.542796
C2,60.610797,47.424080,0.000000,2.218590,35.828348,4.395669,22.498053,0.000000,5.413170,48.083612,...,8.791338,0.0,0.000000,48.996429,0.0,9.795095,0.000000,4.562460,0.0,0.000000
C3,58.255903,60.455497,23.040206,12.120963,55.926653,8.005046,13.657227,14.935300,16.430065,14.594347,...,0.000000,0.0,0.000000,14.871406,0.0,13.378542,0.000000,12.463189,0.0,0.000000
C4,29.917356,6.482332,8.646690,6.065120,10.494273,0.000000,23.064209,18.683413,9.865584,8.215603,...,12.016758,0.0,0.000000,33.486271,0.0,13.388781,0.000000,6.236364,0.0,0.000000
C5,24.500322,53.675826,20.456404,16.142519,14.482668,0.000000,59.112604,8.840272,55.432674,0.000000,...,0.000000,0.0,0.000000,26.407358,0.0,19.797049,57.509813,3.688509,0.0,45.138720


## Load metadata
#### Sample ID: C1-C28, recurrence samples; C29-96, non-recurrence samples

In [8]:
wb = xlrd.open_workbook('data/GSE131512_metaData.xlsx') 
sheet = wb.sheet_by_index(0) 

In [9]:
metadata = []
for i in range(1, sheet.nrows): 
    metadata.append(sheet.row_values(i)) 

metadata_df = pd.DataFrame(metadata, columns=sheet.row_values(0))
metadata_df = metadata_df[metadata_df['Sample name'].str.contains('C')]

def add_recurrence_information(r):
    sample_number = int(r['Sample name'].split('C')[1])
    if sample_number >= 1 and sample_number <= 28:
        return 1
    elif sample_number >= 29:
        return 0

metadata_df['recurrence'] = metadata_df.apply(add_recurrence_information, axis=1)
metadata_df.head()

,Sample name,Age,Gender,recurrence
0,C1,34.443836,Female,1
1,C2,43.353425,Female,1
2,C3,43.852055,Female,1
3,C4,41.421918,Female,1
4,C5,42.115068,Female,1


In [10]:
metadata_df.tail()

,Sample name,Age,Gender,recurrence
91,C92,40.767123,Female,0
92,C93,43.443836,Female,0
93,C94,43.904110,Female,0
94,C95,39.454795,Female,0
95,C96,39.767123,Female,0


## Add recurrence information to TPM table as a multiindex

In [11]:
multi_index_lists = [list(metadata_df['Sample name']), list(metadata_df['recurrence'])]
multi_index_tuples = list(zip(*multi_index_lists))

In [12]:
cancer_tpm.index = pd.MultiIndex.from_tuples(multi_index_tuples, names=['sample', 'recurrence'])

In [13]:
cancer_tpm.to_csv('data/cancer_tpm_multiindexed.tsv', sep='\t', header=True)

KeyboardInterrupt: 

## Example of how to read in the csv with multiindex

In [15]:
cancer_tpm = pd.read_csv('data/cancer_tpm_multiindexed.tsv', sep='\t', index_col=[0,1], header=[0,1])

In [16]:
cancer_tpm.head()

,ENSG,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000283098,ENSG00000283101,ENSG00000283103,ENSG00000283108,ENSG00000283110,ENSG00000283117,ENSG00000283118,ENSG00000283122,ENSG00000283123,ENSG00000283125
,HGNC,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL132857.2,BMS1P21,AC010642.2,AC011451.3,AC141257.4,AC004949.1,AC009951.5,HYMAI,RARRES2P11,AC022726.2
sample,recurrence,,,,,,,,,,,,,,,,,,,,,
C1,1,7.071605,13.279391,0.000000,6.212355,34.038592,9.231355,32.811263,19.136966,12.631348,11.220057,...,12.308474,0.0,4.430132,0.000000,0.0,6.856902,16.599238,22.357146,0.0,19.542796
C2,1,60.610797,47.424080,0.000000,2.218590,35.828348,4.395669,22.498053,0.000000,5.413170,48.083612,...,8.791338,0.0,0.000000,48.996429,0.0,9.795095,0.000000,4.562460,0.0,0.000000
C3,1,58.255903,60.455497,23.040206,12.120963,55.926653,8.005046,13.657227,14.935300,16.430065,14.594347,...,0.000000,0.0,0.000000,14.871406,0.0,13.378542,0.000000,12.463189,0.0,0.000000
C4,1,29.917356,6.482332,8.646690,6.065120,10.494273,0.000000,23.064209,18.683413,9.865584,8.215603,...,12.016758,0.0,0.000000,33.486271,0.0,13.388781,0.000000,6.236364,0.0,0.000000
C5,1,24.500322,53.675826,20.456404,16.142519,14.482668,0.000000,59.112604,8.840272,55.432674,0.000000,...,0.000000,0.0,0.000000,26.407358,0.0,19.797049,57.509813,3.688509,0.0,45.138720


### Example of how to select using multiindex

In [ ]:
cancer_tpm.query('recurrence == 0').head()

In [ ]:
cancer_tpm.query('recurrence == 1').head()

In [ ]:
cancer_tpm.query('sample in ["C1", "C2", "C3"]').head()

In [ ]:
cancer_tpm.iloc[:, cancer_tpm.columns.get_level_values(1).isin(['TSPAN6', 'TNMD', 'DPM1'])].head()

In [ ]:
cancer_tpm.iloc[:, cancer_tpm.columns.get_level_values(0).isin(['ENSG00000000003', 'ENSG00000000005'])].head()

In [ ]:
list(cancer_tpm.columns.get_level_values(0))

In [31]:
cancer_tpm.iloc[:, cancer_tpm.columns.get_level_values(1).isin(['TP53', 'BAK1', 'BBC3', 'BAD'])].query('recurrence == 1').mean()


ENSG             HGNC
ENSG00000002330  BAD      8.915882
ENSG00000030110  BAK1     4.375189
ENSG00000105327  BBC3     5.394735
ENSG00000141510  TP53    14.470466
dtype: float64

In [29]:
cancer_tpm.iloc[:, cancer_tpm.columns.get_level_values(1).isin(['TP53', 'BAK1', 'BBC3', 'BAD'])].query('recurrence == 0').mean()


ENSG             HGNC
ENSG00000002330  BAD      4.694270
ENSG00000030110  BAK1     5.977096
ENSG00000105327  BBC3     5.540076
ENSG00000141510  TP53    12.528913
dtype: float64